In [1]:
import time, math
import torch
from transformers import pipeline, AutoTokenizer

model_name = "yiyanghkust/finbert-tone"
tok = AutoTokenizer.from_pretrained(model_name)

device = 0 if torch.cuda.is_available() else -1
clf = pipeline("sentiment-analysis", model=model_name, tokenizer=tok, device=device)

texts = news_df["Headlines"].astype(str).tolist()
n = min(2000, len(texts))      # sample size for timing
bs = 32                        # try 16/32/64 on GPU, 8/16 on CPU

# warm-up
_ = clf(texts[:min(64, n)], top_k=None, truncation=True, batch_size=bs)

t0 = time.time()
_ = clf(texts[:n], top_k=None, truncation=True, batch_size=bs)
t1 = time.time()

throughput = n / (t1 - t0)     # headlines per second
print(f"Throughput ~ {throughput:.1f} headlines/sec (batch_size={bs}, device={'GPU' if device>=0 else 'CPU'})")

# crude extrapolation (no promises, just math)
total = len(texts)
est_seconds = total / throughput
print(f"Estimated total seconds for all {total} headlines: {est_seconds:.0f} (rough)")


KeyboardInterrupt: 